# Chapter 1

<a target="_blank" href="https://colab.research.google.com/github/wandb/edu/blob/main/rag-advanced/notebooks/Chapter01.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!--- @wandbcode{rag-course-01} -->

In this chapter, we will learn about building a simple RAG pipeline. We will mainly focus on how to preprocess and chunk the data followed by building a simple retrieval engine without using any fancy "Vector Index". The idea is to show the inner working of a retrieval pipeline and make you understand the workflow from a user query to a generated response using an LLM.

For this chapter you will need a Cohere API key and a Weights & Biases account. Run the next cell to pull the github repo and install the dependencies.

In [1]:
from rich import print

In [2]:
import nltk

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/elshehawy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Now we are ready to start the chapter.

We will start by logging into Weights & Biases and setup the API keys in the environment. You should be able to get your Cohere API key from [here](https://dashboard.cohere.com/api-keys).

In [3]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: elshehawy. Use `wandb login --relogin` to force relogin


True

In [4]:
import pathlib
from typing import List

import weave

from scripts.utils import display_source

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/elshehawy/.config/sagemaker/config.yaml


Here, we will start a Weights & Biases run. We will be using this to download a [W&B Artifact](https://docs.wandb.ai/guides/artifacts) called `wandb_docs`. This is the raw Weights & Biases documentation. W&B Artifacts is suited for versiong different data sources which needs preprocessing/cleaning.

In [5]:
WANDB_PROJECT = "rag-course"

run = wandb.init(
    project=WANDB_PROJECT,
    group="Chapter 1",
)

## Data ingestion

### Loading the data

Use [W&B Artifacts](https://docs.wandb.ai/guides/artifacts) to track and version data as the inputs and outputs of your W&B Runs. For example, a model training run might take in a dataset as input and produce a trained model as output. W&B Artifacts is a powerful object storage with rich UI functionalities.

Below we are downloading an artifact named `wandb_docs` which will download 400 odd markdown files in your `./data/wandb_docs` directory. This is will be our data source.

In [6]:
documents_artifact = run.use_artifact(
    f"rag-course/dev/wandb_docs:latest", type="dataset"
)
data_dir = "./data/wandb_docs"

docs_dir = documents_artifact.download(data_dir)

wandb:   406 of 406 files downloaded.  


Let's inspect the `./data/wandb_docs` directory and look at the names of the first 5 files. We should see that they are all in markdown (`.md`) file format.

In [7]:
docs_dir = pathlib.Path(docs_dir)
docs_files = sorted(docs_dir.rglob("*.md"))

print(f"Number of files: {len(docs_files)}\n")
print("First 5 files:\n{files}".format(files="\n".join(map(str, docs_files[:5]))))

Number of files: 406

First 5 files:
data/wandb_docs/guides/app/features/anon.md
data/wandb_docs/guides/app/features/custom-charts/intro.md
data/wandb_docs/guides/app/features/custom-charts/walkthrough.md
data/wandb_docs/guides/app/features/intro.md
data/wandb_docs/guides/app/features/notes.md

Lets look at an example file. We can take the first element of the list (`docs_files`) and use the `Path.read_text` method to get the decoded contents of the file as a string.

In [8]:
print(docs_files[0].read_text())

---
description: Log and visualize data without a W&B account
displayed_sidebar: default
---

# Anonymous Mode

Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your 
code, see a W&B dashboard, and visualize results without needing to create a W&B account first.

Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`

:::info
**Publishing a paper?** Please (https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions
about how to make your code accessible while using W&B, reach out to us at support@wandb.com.
:::

### How does someone without an account see results?

If someone runs your script and you have to set `anonymous="allow"`:

1. **Auto-create temporary account:** W&B checks for an account that's already signed in. If there's no account, we
automatically create a new anonymous account and save that API key for the session.
2. **Log results quickly:** The user can run and re-run the script, and automatically see results show up in the 
W&B dashboard UI. These unclaimed anonymous runs will be available for 7 days.
3. **Claim data when it's useful**: Once the user finds valuable results in W&B, they can easily click a button in 
the banner at the top of the page to save their run data to a real account. If they don't claim a run, it will be 
deleted after 7 days.

:::caution
**Anonymous run links are sensitive**. These links allow anyone to view and claim the results of an experiment for 
7 days, so make sure to only share links with people you trust. If you're trying to share results publicly, but 
hide the author's identity, please contact us at support@wandb.com to share more about your use case.
:::

### What happens to users with existing accounts?

If you set `anonymous="allow"` in your script, we will check to make sure there's not an existing account first, 
before creating an anonymous account. This means that if a W&B user finds your script and runs it, their results 
will be logged correctly to their account, just like a normal run.

### What are features that aren't available to anonymous users?

*   **No persistent data**: Runs are only saved for 7 days in an anonymous account. Users can claim anonymous run 
data by saving it to a real account.


![](@site/static/images/app_ui/anon_mode_no_data.png)

*   **No artifact logging**: Runs will print a warning on the command line that you can't log an artifact to an 
anonymous run.

![](@site/static/images/app_ui/anon_example_warning.png)

* **No profile or settings pages**: Certain pages aren't available in the UI, because they're only useful for real 
accounts.

## Example usage

[Try the example notebook](http://bit.ly/anon-mode) to see how anonymous mode works.

```python
import wandb

# Start a run allowing anonymous accounts
wandb.init(anonymous="allow")

# Log results from your training loop
wandb.log({"acc": 0.91})

# Mark the run as finished
wandb.finish()
```

💡 Looking at the example, we see some structure and format to it.

It is always a good practice to look through few examples to see if there is any pattern to your data source.
It helps to come up with better preprocessing steps and chunking strategies.

Now, let's store our documents as dictionaries with content (raw text) and metadata.

Metadata is extra information for that data point which can be used to group together similar data points, or filter out a few data points.
We will see in future chapters the importance of metadata and why it should not be ignored while building the ingestion pipeline.

The metadata can be derived (`raw_tokens`) or is inherent (`source`) to the data point.

Note that we are simply doing word counting and calling it `raw_tokens`.
In practice we would be using a [tokenizer](https://docs.cohere.com/docs/tokens-and-tokenizers) to calculate the token counts but this naive calculation is an okay approximation for now.

In [9]:
# We'll store the files as dictionaries with some content and metadata
data = []
for file in docs_files:
    content = file.read_text()
    data.append(
        {
            "content": content,
            "metadata": {
                "source": str(file.relative_to(docs_dir)),
                "raw_tokens": len(content.split()),
            },
        }
    )
print(data[0])

{
    'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: 
default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use 
Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a
W&B account first.\n\nAllow results to be logged in Anonymous Mode with 
`wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite 
W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your 
code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an 
account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create 
temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically 
create a new anonymous account and save that API key for the session.\n2. **Log results quickly:** The user can run
and re-run the script, and automatically see results show up in the W&B dashboard UI. These unclaimed anonymous 
runs will be available for 7 days.\n3. **Claim data when it\'s useful**: Once the user finds valuable results in 
W&B, they can easily click a button in the banner at the top of the page to save their run data to a real account. 
If they don\'t claim a run, it will be deleted after 7 days.\n\n:::caution\n**Anonymous run links are sensitive**. 
These links allow anyone to view and claim the results of an experiment for 7 days, so make sure to only share 
links with people you trust. If you\'re trying to share results publicly, but hide the author\'s identity, please 
contact us at support@wandb.com to share more about your use case.\n:::\n\n### What happens to users with existing 
accounts?\n\nIf you set `anonymous="allow"` in your script, we will check to make sure there\'s not an existing 
account first, before creating an anonymous account. This means that if a W&B user finds your script and runs it, 
their results will be logged correctly to their account, just like a normal run.\n\n### What are features that 
aren\'t available to anonymous users?\n\n*   **No persistent data**: Runs are only saved for 7 days in an anonymous
account. Users can claim anonymous run data by saving it to a real 
account.\n\n\n![](@site/static/images/app_ui/anon_mode_no_data.png)\n\n*   **No artifact logging**: Runs will print
a warning on the command line that you can\'t log an artifact to an anonymous 
run.\n\n![](@site/static/images/app_ui/anon_example_warning.png)\n\n* **No profile or settings pages**: Certain 
pages aren\'t available in the UI, because they\'re only useful for real accounts.\n\n## Example usage\n\n[Try the 
example notebook](http://bit.ly/anon-mode) to see how anonymous mode works.\n\n```python\nimport wandb\n\n# Start a
run allowing anonymous accounts\nwandb.init(anonymous="allow")\n\n# Log results from your training 
loop\nwandb.log({"acc": 0.91})\n\n# Mark the run as finished\nwandb.finish()\n```\n',
    'metadata': {'source': 'guides/app/features/anon.md', 'raw_tokens': 470}
}

Checking the total number of tokens of your data source is a good practice. In this case, the total tokens is more than 200k. Surely, most LLM providers cannot process this many tokens. Building a RAG is justified in such cases.

In [10]:
total_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], data))
print(f"Total Tokens in dataset: {total_tokens}")

Total Tokens in dataset: 260530

## W&B Weave

In the previous section we used W&B Artifacts to download the source documents.

We could have used it to log our processed data but instead we will use W&B Weave for the task.

Why?

- W&B Weave is standalone and doesn't need backward compatibility with core Weights & Biases offerings.
- W&B Weave is designed for modern LLMOps use case.
- We would like to keep the data, "models" (function, API, collection of parser, extrators, and more) and evaluators in "one single source of truth" managed by W&B Weave.

What?

W&B Weave is a lightweight toolkit for tracking and evaluating LLM applications:

- Log and debug language model inputs, outputs, and traces
- Build rigorous, apples-to-apples evaluations for language model use cases
- Organize all the information generated across the LLM workflow, from experimentation to evaluations to production


The newly created list of dictionaries with `content` and `metadata` will now be logged as a W&B Weave Dataset called `raw_data`. Notice that our processed data is a list of dicts.

Let's initialize W&B Weave. Once intialized, we will start tracking (more on it later) the inputs and the outputs along with underlying attributes (model name, top_k, etc.).

In [11]:
WANDB_PROJECT = "rag-course"

weave_client = weave.init(WANDB_PROJECT)

Logged in as Weights & Biases user: elshehawy.
View Weave data at https://wandb.ai/elshehawy/rag-course/weave


In [12]:
# build weave dataset
raw_data = weave.Dataset(name="raw_data", rows=data)

# publish the dataset
weave.publish(raw_data)

📦 Published to https://wandb.ai/elshehawy/rag-course/weave/objects/raw_data/versions/5gpOPHrLiBuPtOYa3bn1yKfcrYnxXCA8JtN9xYXkxkg


ObjectRef(entity='elshehawy', project='rag-course', name='raw_data', _digest='5gpOPHrLiBuPtOYa3bn1yKfcrYnxXCA8JtN9xYXkxkg', _extra=())

![raw_data](../images/01_raw_data.png)

### Chunking the data

Each document contains a large number of tokens, so we need to split it into smaller chunks to manage the number of tokens per chunk. This approach serves three main purposes:

* Most embedding models have a limit of 512 tokens per input (based on their training data and parameters).

* Chunking allows us to retrieve and send only the most relevant portions to our LLM, significantly reducing the total token count. This helps keep the LLM's cost and processing time manageable.

* When the text is small-sized, embedding models tend to generate better vectors as they can capture more fine-grained details and nuances in the text, resulting in more accurate representations.

When choosing chunk size, consider these trade-offs:

- Smaller chunks (100-200 tokens):
  * More precise retrieval
  * Better for finding specific details
  * May lack broader context

- Larger chunks (500-1000 tokens):
  * Provide more context
  * Capture more coherent ideas
  * May introduce noise and reduce precision

The optimal size depends on your data, expected queries, and model capabilities. Experiment with different sizes to find the best balance for your use case.

Here we are chunking each content (text) to a maximum length of 300 tokens (`CHUNK_SIZE`).
For now, we will not be overlapping (`CHUNK_OVERLAP`) the content of one chunk with another chunk.

In [13]:
# These are hyperparameters of our ingestion pipeline

CHUNK_SIZE = 300
CHUNK_OVERLAP = 0


def split_into_chunks(
    text: str, chunk_size: int = CHUNK_SIZE, chunk_overlap: int = CHUNK_OVERLAP
) -> List[str]:
    """Function to split the text into chunks of a maximum number of tokens
    ensure that the chunks are of size CHUNK_SIZE and overlap by chunk_overlap tokens
    use the `tokenizer.encode` method to tokenize the text
    """
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start = end - chunk_overlap
    return chunks

We will use the `raw_data` artifact we just published as W&B Weave Dataset as the input to the chunking function.

In [14]:
# download the `raw_data` Dataset
# raw_data = weave.ref(
#     "weave:///rag-course/rag-course/object/raw_data:nuZosGsP58MXKxhluN2hzvKK9XB8xSxlTuOBmEzWzLo"
# ).get()
# uncomment the next line to get the raw data from weave from your own project instead
raw_data = weave.ref("raw_data:v0").get()

# this is how we index into the data
print(raw_data.rows[0])

WeaveDict({'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: 
default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use 
Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a
W&B account first.\n\nAllow results to be logged in Anonymous Mode with 
`wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite 
W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your 
code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an 
account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create 
temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically 
create a new anonymous account and save that API key for the session.\n2. **Log results quickly:** The user can run
and re-run the script, and automatically see results show up in the W&B dashboard UI. These unclaimed anonymous 
runs will be available for 7 days.\n3. **Claim data when it\'s useful**: Once the user finds valuable results in 
W&B, they can easily click a button in the banner at the top of the page to save their run data to a real account. 
If they don\'t claim a run, it will be deleted after 7 days.\n\n:::caution\n**Anonymous run links are sensitive**. 
These links allow anyone to view and claim the results of an experiment for 7 days, so make sure to only share 
links with people you trust. If you\'re trying to share results publicly, but hide the author\'s identity, please 
contact us at support@wandb.com to share more about your use case.\n:::\n\n### What happens to users with existing 
accounts?\n\nIf you set `anonymous="allow"` in your script, we will check to make sure there\'s not an existing 
account first, before creating an anonymous account. This means that if a W&B user finds your script and runs it, 
their results will be logged correctly to their account, just like a normal run.\n\n### What are features that 
aren\'t available to anonymous users?\n\n*   **No persistent data**: Runs are only saved for 7 days in an anonymous
account. Users can claim anonymous run data by saving it to a real 
account.\n\n\n![](@site/static/images/app_ui/anon_mode_no_data.png)\n\n*   **No artifact logging**: Runs will print
a warning on the command line that you can\'t log an artifact to an anonymous 
run.\n\n![](@site/static/images/app_ui/anon_example_warning.png)\n\n* **No profile or settings pages**: Certain 
pages aren\'t available in the UI, because they\'re only useful for real accounts.\n\n## Example usage\n\n[Try the 
example notebook](http://bit.ly/anon-mode) to see how anonymous mode works.\n\n```python\nimport wandb\n\n# Start a
run allowing anonymous accounts\nwandb.init(anonymous="allow")\n\n# Log results from your training 
loop\nwandb.log({"acc": 0.91})\n\n# Mark the run as finished\nwandb.finish()\n```\n', 'metadata': {'source': 
'guides/app/features/anon.md', 'raw_tokens': 470}})

A Weave Dataset is compatible with the rest of the Weave workflow and is pythonic.

Let us chunk each document in the raw data Artifact. We create a new list of dictionaries with the chuked text (`content`) and with `metadata`.

In [15]:
chunked_data = []
for doc in raw_data.rows:
    chunks = split_into_chunks(doc["content"])
    for chunk in chunks:
        chunked_data.append(
            {
                "content": chunk,
                "metadata": {
                    "source": doc["metadata"]["source"],
                    "raw_tokens": len(chunk.split()),
                },
            }
        )

print(chunked_data[:2])

[
    {
        'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # 
Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let 
someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. 
Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a 
paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions 
about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does 
someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. 
**Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we 
automatically create a new anonymous account and save that API key for the session. 2. **Log results quickly:** The
user can run and re-run the script, and automatically see results show up in the W&B dashboard UI. These unclaimed 
anonymous runs will be available for 7 days. 3. **Claim data when it\'s useful**: Once the user finds valuable 
results in W&B, they can easily click a button in the banner at the top of the page to save their run data to a 
real account. If they don\'t claim a run, it will be deleted after 7 days. :::caution **Anonymous run links are 
sensitive**. These links allow anyone to view and claim the results of an experiment for 7 days, so make sure to 
only share links with people you trust. If you\'re trying to share results publicly, but hide the author\'s 
identity, please contact us at support@wandb.com to share more about your use case. ::: ### What happens to users 
with existing accounts? If you set `anonymous="allow"` in',
        'metadata': {'source': 'guides/app/features/anon.md', 'raw_tokens': 300}
    },
    {
        'content': 'your script, we will check to make sure there\'s not an existing account first, before creating
an anonymous account. This means that if a W&B user finds your script and runs it, their results will be logged 
correctly to their account, just like a normal run. ### What are features that aren\'t available to anonymous 
users? * **No persistent data**: Runs are only saved for 7 days in an anonymous account. Users can claim anonymous 
run data by saving it to a real account. ![](@site/static/images/app_ui/anon_mode_no_data.png) * **No artifact 
logging**: Runs will print a warning on the command line that you can\'t log an artifact to an anonymous run. 
![](@site/static/images/app_ui/anon_example_warning.png) * **No profile or settings pages**: Certain pages aren\'t 
available in the UI, because they\'re only useful for real accounts. ## Example usage [Try the example 
notebook](http://bit.ly/anon-mode) to see how anonymous mode works. ```python import wandb # Start a run allowing 
anonymous accounts wandb.init(anonymous="allow") # Log results from your training loop wandb.log({"acc": 0.91}) # 
Mark the run as finished wandb.finish() ```',
        'metadata': {'source': 'guides/app/features/anon.md', 'raw_tokens': 170}
    }
]

### Cleaning the data

Cleaning up data is crucial for most AI pipelines and applies to a RAG/Agentic pipeline as well.
Usually, higher quality chunks provided to an LLM generates a higher quality response.

here, it is particularly important for several reasons:

- **Tokenization issues**: Special tokens can interfere with the model's tokenization process, leading to unexpected interpretations of the text.
- **Vocabulary pollution**: Unusual tokens can inflate the model's vocabulary, potentially diluting the importance of meaningful terms.
- **Semantic distortion**: Special characters or formatting tokens (e.g., HTML tags) can alter the semantic meaning of sentences if not properly handled.
- **Consistency**: Removing or standardizing special tokens ensures consistent representation across different data sources.
- **Model efficiency**: Cleaner data often leads to more efficient model training and inference, as the model doesn't need to process irrelevant tokens.

By cleaning the data, we ensure that our model focuses on the meaningful content rather than artifacts or noise in the text.

We will use a simple function `make_text_tokenization_safe` to remove special tokens from the text.
This is a good practice as most LLM providers do not like special tokens in the text.

In [16]:
from scripts.preprocess import make_text_tokenization_safe

display_source(make_text_tokenization_safe)

   1 def make_text_tokenization_safe(                                                                              
   2     content: str, special_tokens_set: set = get_special_tokens_set()                                          
   3 ) -> str:                                                                                                     
   4     """                                                                                                       
   5     Makes the text safe for tokenization by removing special tokens.                                          
   6                                                                                                               
   7     Args:                                                                                                     
   8         content: A string containing the text to be processed.                                                
   9         special_tokens_set: A set of special tokens to be removed from the text.                              
  10                                                                                                               
  11     Returns:                                                                                                  
  12         A string with the special tokens removed.                                                             
  13     """                                                                                                       
  14                                                                                                               
  15     def remove_special_tokens(text: str) -> str:                                                              
  16         """                                                                                                   
  17         Removes special tokens from the given text.                                                           
  18                                                                                                               
  19         Args:                                                                                                 
  20             text: A string representing the text.                                                             
  21                                                                                                               
  22         Returns:                                                                                              
  23             The text with special tokens removed.                                                             
  24         """                                                                                                   
  25         for token in special_tokens_set:                                                                      
  26             text = text.replace(token, "")                                                                    
  27         return text                                                                                           
  28                                                                                                               
  29     cleaned_content = remove_special_tokens(content)                                                          
  30     return cleaned_content                                                                                    
  31                                                                                                               

In [17]:
cleaned_data = []
for doc in chunked_data:
    cleaned_doc = doc.copy()
    cleaned_doc["cleaned_content"] = make_text_tokenization_safe(doc["content"])
    cleaned_doc["metadata"]["cleaned_tokens"] = len(
        cleaned_doc["cleaned_content"].split()
    )
    cleaned_data.append(cleaned_doc)
print(cleaned_data[:2])

[
    {
        'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # 
Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let 
someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. 
Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a 
paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions 
about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does 
someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. 
**Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we 
automatically create a new anonymous account and save that API key for the session. 2. **Log results quickly:** The
user can run and re-run the script, and automatically see results show up in the W&B dashboard UI. These unclaimed 
anonymous runs will be available for 7 days. 3. **Claim data when it\'s useful**: Once the user finds valuable 
results in W&B, they can easily click a button in the banner at the top of the page to save their run data to a 
real account. If they don\'t claim a run, it will be deleted after 7 days. :::caution **Anonymous run links are 
sensitive**. These links allow anyone to view and claim the results of an experiment for 7 days, so make sure to 
only share links with people you trust. If you\'re trying to share results publicly, but hide the author\'s 
identity, please contact us at support@wandb.com to share more about your use case. ::: ### What happens to users 
with existing accounts? If you set `anonymous="allow"` in',
        'metadata': {'source': 'guides/app/features/anon.md', 'raw_tokens': 300, 'cleaned_tokens': 300},
        'cleaned_content': '--- description: Log and visualize data without a W&B account displayed_sidebar: 
default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous 
Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B 
account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info 
**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have
questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How
does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 
1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, 
we automatically create a new anonymous account and save that API key for the session. 2. **Log results quickly:** 
The user can run and re-run the script, and automatically see results show up in the W&B dashboard UI. These 
unclaimed anonymous runs will be available for 7 days. 3. **Claim data when it\'s useful**: Once the user finds 
valuable results in W&B, they can easily click a button in the banner at the top of the page to save their run data
to a real account. If they don\'t claim a run, it will be deleted after 7 days. :::caution **Anonymous run links 
are sensitive**. These links allow anyone to view and claim the results of an experiment for 7 days, so make sure 
to only share links with people you trust. If you\'re trying to share results publicly, but hide the author\'s 
identity, please contact us at support@wandb.com to share more about your use case. ::: ### What happens to users 
with existing accounts? If you set `anonymous="allow"` in'
    },
    {
        'content': 'your script, we will check to make sure there\'s not an existing account first, before creati

Again we will store the cleaned data as a Weave Dataset named `chunked_data`.

In [18]:
dataset = weave.Dataset(name="chunked_data", rows=cleaned_data)
weave.publish(dataset)

📦 Published to https://wandb.ai/elshehawy/rag-course/weave/objects/chunked_data/versions/grMsWPBotYOknW2YYt6PnO9131x9766BTQ3KrhAHcpk


ObjectRef(entity='elshehawy', project='rag-course', name='chunked_data', _digest='grMsWPBotYOknW2YYt6PnO9131x9766BTQ3KrhAHcpk', _extra=())

![chunked_data](../images/01_chunked_data.png)

## Vectorizing the data

One of the key ingredient of most retrieval systems is to represent the given modality (text in our case) as a vector.

This vector is a numerical representation representing the "content" of that modality (text).

Text vectorization (text to vector) can be done using various techniques like [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model), [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (Term Frequency-Inverse Document Frequency), and embeddings like [Word2Vec](https://en.wikipedia.org/wiki/Word2vec), [GloVe](https://nlp.stanford.edu/projects/glove/), and transformer based architectures like BERT and more, which capture the semantic meaning and relationships between words or sentences.


In this chapter, we'll use TF-IDF (Term Frequency-Inverse Document Frequency) for vectorizing our contents. Here's why:

- **Simplicity**: TF-IDF is straightforward to implement and understand, making it an excellent starting point for RAG systems.
- **Efficiency**: It's computationally lightweight, allowing for quick processing of large document collections.
- **No training required**: Unlike embedding models, TF-IDF doesn't need pre-training, making it easy to get started quickly.
- **Interpretability**: The resulting vectors are directly related to word frequencies, making them easy to interpret.

While more advanced methods like embeddings often provide better performance, especially for semantic understanding, we'll explore these in later chapters as we progress through the course.

Let's download the `cleaned_data` artifact and use it to generate vectors for our chunks.

In [19]:
# chunked_data = weave.ref(
#     "weave:///rag-course/rag-course/object/chunked_data:Lt6M8qCUICD1JZTlMYzuDLTVtvFYESxvj3tcAIoPrtE"
# ).get()
# uncomment the next line to get the chunked data from weave from your own project instead
chunked_data = weave.ref("chunked_data:v0").get()
print(chunked_data.rows[:2])

[
    WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default
--- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to 
let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account 
first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info 
**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have
questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How
does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 
1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, 
we automatically create a new anonymous account and save that API key for the session. 2. **Log results quickly:** 
The user can run and re-run the script, and automatically see results show up in the W&B dashboard UI. These 
unclaimed anonymous runs will be available for 7 days. 3. **Claim data when it\'s useful**: Once the user finds 
valuable results in W&B, they can easily click a button in the banner at the top of the page to save their run data
to a real account. If they don\'t claim a run, it will be deleted after 7 days. :::caution **Anonymous run links 
are sensitive**. These links allow anyone to view and claim the results of an experiment for 7 days, so make sure 
to only share links with people you trust. If you\'re trying to share results publicly, but hide the author\'s 
identity, please contact us at support@wandb.com to share more about your use case. ::: ### What happens to users 
with existing accounts? If you set `anonymous="allow"` in', 'metadata': {'source': 'guides/app/features/anon.md', 
'raw_tokens': 300, 'cleaned_tokens': 300}, 'cleaned_content': '--- description: Log and visualize data without a 
W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able
to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without 
needing to create a W&B account first. Allow results to be logged in Anonymous Mode with 
`wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite 
W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your 
code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account 
see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary 
account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a 
new anonymous account and save that API key for the session. 2. **Log results quickly:** The user can run and 
re-run the script, and automatically see results show up in the W&B dashboard UI. These unclaimed anonymous runs 
will be available for 7 days. 3. **Claim data when it\'s useful**: Once the user finds valuable results in W&B, 
they can easily click a button in the banner at the top of the page to save their run data to a real account. If 
they don\'t claim a run, it will be deleted after 7 days. :::caution **Anonymous run links are sensitive**. These 
links allow anyone to view and claim the results of an experiment for 7 days, so make sure to only share links with
people you trust. If you\'re trying to share results publicly, but hide the author\'s identity, please contact us 
at support@wandb.com to share more about your use case. ::: ### What happens to users with existing accounts? If 
you set `anonymous="allow"` in'}),
    WeaveDict({'content': 'your script, we will check to make sure there\'s not an existing account first, before 
creating an anonymous a

Next, we will create a simple `Retriever` class. This class is responsible for vectorizing the chunks using the `index_data` method and provides a convenient method `search`, for querying the vector index using cosine distance similarity.

- `index_data` will take a list of chunks and vectorize it using TF-IDF and store it as `index`.

- `search` will take a `query` (question) and vectorize it using the same technique (TF-IDF in our case). It then computes the cosine distance between the query vector and the index (list of vectors) and pick the top `k` vectors from the index. These top `k` vectors represent the chunks that are closest (most relevant) to the `query`.

---

Note that the `Retriever` class is inherited from `weave.Model`.

A Model is a combination of data (which can include configuration, trained model weights, or other information) and code that defines how the model operates. By structuring your code to be compatible with this API, you benefit from a structured way to version your application so you can more systematically keep track of your experiments.

To create a model in Weave, you need the following:

- a class that inherits from weave.Model
- type definitions on all attributes
- a typed `predict`, `infer` or `forward` method with `@weave.op()` decorator.

Imagine `weave.op()` to be a drop in replacement for `print` or logging statement.
However, it does a lot more than just printing and logging by tracking the inputs and outputs of the function and storing them as Weave objects. In addition to state tracking you also get a nice weave UI to inspect the inputs, outputs, and other metadata.

If you have not initialized a weave run by doing `weave.init`, the code will work as it is without any tracking.

The `predict` method decorated with `weave.op()` will track the model settings along with the inputs and outputs anytime you call it.

In [20]:
from scripts.retriever import TFIDFRetriever

display_source(TFIDFRetriever)

/home/elshehawy/anaconda3/envs/ragpp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   1 class TFIDFRetriever(weave.Model):                                                                            
   2     """                                                                                                       
   3     A retriever model that uses TF-IDF for indexing and searching documents.                                  
   4                                                                                                               
   5     Attributes:                                                                                               
   6         vectorizer (TfidfVectorizer): The TF-IDF vectorizer.                                                  
   7         index (list): The indexed data.                                                                       
   8         data (list): The data to be indexed.                                                                  
   9     """                                                                                                       
  10                                                                                                               
  11     vectorizer: TfidfVectorizer = TfidfVectorizer()                                                           
  12     index: list = None                                                                                        
  13     data: list = None                                                                                         
  14                                                                                                               
  15     def index_data(self, data):                                                                               
  16         """                                                                                                   
  17         Indexes the provided data using TF-IDF.                                                               
  18                                                                                                               
  19         Args:                                                                                                 
  20             data (list): A list of documents to be indexed. Each document should be a dictionary              
  21                          containing a key 'cleaned_content' with the text to be indexed.                      
  22         """                                                                                                   
  23         self.data = data                                                                                      
  24         docs = [doc["cleaned_content"] for doc in data]                                                       
  25         self.index = self.vectorizer.fit_transform(docs)                                                      
  26                                                                                                               
  27     @weave.op()                                                                                               
  28     def search(self, query, k=5):                                                                             
  29         """                                                                                                   
  30         Searches the indexed data for the given query using cosine similarity.                                
  31                                                                                                               
  32         Args:                                                                                                 
  33             query (str): The search query.                                                                    
  34             k (int): The number of top results to return. Default is 5.                                       
  35                                                    

This simple TF-IDF based retriever serves as a good starting point, but for more complex applications, it can be extended or improved in several ways:

- **Semantic search**: Implement embedding-based retrieval using dense vectors for better semantic understanding.
- **Hybrid retrieval**: Combine TF-IDF with embedding-based methods to balance lexical and semantic matching.
- **Query expansion**: Incorporate techniques like automatic query expansion to improve recall.
- **Document ranking**: Implement more sophisticated ranking algorithms, such as BM25 and re-ranking models.
- **Scalability**: For larger datasets, consider using approximate nearest neighbor search techniques or vector databases.

As we progress through the course, we'll explore some of these concepts and gradually enhance our retriever to make it more robust and efficient for real-world applications.

Now, let's see our `TFIDFRetriever` in action. We will index our chunked data and then query the retriever to fetch related chunks from the index.

In [21]:
retriever = TFIDFRetriever()
retriever.index_data(list(map(dict, chunked_data.rows)))

In [22]:
query = "How do I use W&B to log metrics in my training script?"
search_results = retriever.search(query)
for result in search_results:
    print(result)

🍩 https://wandb.ai/elshehawy/rag-course/r/call/0192cfa3-326d-7a60-aa26-7035e5f3be94


{
    'source': 'guides/technical-faq/general.md',
    'text': '--- displayed_sidebar: default --- # General ### What does `wandb.init` do to my training process? 
When `wandb.init()` is called from your training script an API call is made to create a run object on our servers. 
A new process is started to stream and collect metrics, thereby keeping all threads and logic out of your primary 
process. Your script runs normally and writes to local files, while the separate process streams them to our 
servers along with system metrics. You can always turn off streaming by running `wandb off` from your training 
directory, or setting the `WANDB_MODE` environment variable to `offline`. ### Does your tool track or store 
training data? You can pass a SHA or other unique identifier to `wandb.config.update(...)` to associate a dataset 
with a training run. W&B does not store any data unless `wandb.save` is called with the local file name. ### What 
formula do you use for your smoothing algorithm? We use the same exponential moving average formula as TensorBoard.
You can find an [explanation 
here](https://stackoverflow.com/questions/42281844/what-is-the-mathematics-behind-the-smoothing-parameter-in-tensor
boards-scalar). ### How do I get the random run name in my script? Call `wandb.run.save()` and then get the name 
with `wandb.run.name` . ### What is the difference between `.log()` and `.summary`? The summary is the value that 
shows in the table while the log will save all the values for plotting later. For example, you might want to call 
`wandb.log` every time the accuracy changes. Usually, you can just use .log. `wandb.log()` will also update the 
summary value by default unless you have set the summary manually for that metric The scatterplot and parallel 
coordinate plots will also use the summary value while the line plot plots all of the values set by .log The reason
we have both is that some people like to set the summary manually because',
    'score': 0.28870928597000667
}

{
    'source': 'guides/technical-faq/troubleshooting.md',
    'text': '--- displayed_sidebar: default --- import Tabs from \'@theme/Tabs\'; import TabItem from 
\'@theme/TabItem\'; # Troubleshooting ### If wandb crashes, will it possibly crash my training run? It is extremely
important to us that we never interfere with your training runs. We run wandb in a separate process to make sure 
that if wandb somehow crashes, your training will continue to run. If the internet goes out, wandb will continue to
retry sending data to [wandb.ai](https://wandb.ai). ### Why is a run marked crashed in W&B when it’s training fine 
locally? This is likely a connection problem — if your server loses internet access and data stops syncing to W&B, 
we mark the run as crashed after a short period of retrying. ### Does logging block my training? "Is the logging 
function lazy? I don\'t want to be dependent on the network to send the results to your servers and then carry on 
with my local operations." Calling `wandb.log` writes a line to a local file; it does not block any network calls. 
When you call `wandb.init` we launch a new process on the same machine that listens for filesystem changes and 
talks to our web service asynchronously from your training process. ### How do I stop wandb from writing to my 
terminal or my jupyter notebook output? Set the environment variable 
[`WANDB_SILENT`](../track/environment-variables.md) to `true`. <Tabs defaultValue="python" values={[ {label: 
\'Python\', value: \'python\'}, {label: \'Jupyter Notebook\', value: \'notebook\'}, {label: \'Command Line\', 
value: \'command-line\'}, ]}> <TabItem value="python"> ```python os.environ["WANDB_SILENT"] = "true" ``` </TabItem>
<TabItem value="notebook"> ```python %env WANDB_SILENT=true ``` </TabItem> <TabItem value="command-line"> ```python
WANDB_SILENT=true ``` </TabItem> </Tabs> ### How do I kill a job with wandb? Press `Ctrl+D` on your keyboard to 
stop a script that is instrumented with wandb. ### How do I deal with network issues? If you\'re seeing SSL or 
network errors:`wandb:',
    'score': 0.25915729952187805
}

{
    'source': 'guides/track/log/logging-faqs.md',
    'text': '--- description: >- Answers to frequently asked questions about tracking data from machine learning 
experiments with W&B Experiments. displayed_sidebar: default --- import Tabs from \'@theme/Tabs\'; import TabItem 
from \'@theme/TabItem\'; # Logging FAQs <head> <title>Frequently Asked Questions About Logging Data from 
Experiments</title> </head> ### How can I organize my logged charts and media in the W&B UI? We treat `/` as a 
separator for organizing logged panels in the W&B UI. By default, the component of the logged item\'s name before a
`/` is used to define a group of panel called a "Panel Section". ```python wandb.log({"val/loss": 1.1, "val/acc": 
0.3}) wandb.log({"train/loss": 0.1, "train/acc": 0.94}) ``` In the [Workspace](../../app/pages/workspaces.md) 
settings, you can change whether panels are grouped by just the first component or by all components separated by 
`/`. ### How can I compare images or media across epochs or steps? Each time you log images from a step, we save 
them to show in the UI. Expand the image panel, and use the step slider to look at images from different steps. 
This makes it easy to compare how a model\'s output changes during training. ### What if I want to log some metrics
on batches and some metrics only on epochs? If you\'d like to log certain metrics in every batch and standardize 
plots, you can log x axis values that you want to plot with your metrics. Then in the custom plots, click edit and 
select a custom x-axis. ```python wandb.log({"batch": batch_idx, "loss": 0.3}) wandb.log({"epoch": epoch, 
"val_acc": 0.94}) ``` ### How do I log a list of values? <!-- Logging lists directly is not supported. Instead, 
list-like collections of numerical data are converted to [histograms](../../../ref/python/data-types/histogram.md).
To log all of the entries in a list, give a name to each entry in the list and use those names as keys',
    'score': 0.25378170864192895
}

{
    'source': 'guides/track/log/media.md',
    'text': 'table = wandb.Table(columns=columns) table.add_data("I love my phone", "1", "1") table.add_data("My 
phone sucks", "0", "-1") wandb.log({"examples": table}) ``` You can also pass a pandas `DataFrame` object. 
```python table = wandb.Table(dataframe=my_dataframe) ``` </TabItem> <TabItem value="html"> ```python 
wandb.log({"custom_file": wandb.Html(open("some.html"))}) wandb.log({"custom_string": wandb.Html(\'<a 
href="https://mysite">Link</a>\')}) ``` Custom html can be logged at any key, and this exposes an HTML panel on the
run page. By default we inject default styles, you can disable default styles by passing `inject=False`. ```python 
wandb.log({"custom_file": wandb.Html(open("some.html"), inject=False)}) ``` </TabItem> </Tabs> ## Frequently Asked 
Questions ### How can I compare images or media across epochs or steps? Each time you log images from a step, we 
save them to show in the UI. Expand the image panel, and use the step slider to look at images from different 
steps. This makes it easy to compare how a model\'s output changes during training. ### What if I want to integrate
W&B into my project, but I don\'t want to upload any images or media? W&B can be used even for projects that only 
log scalars — you specify any files or data you\'d like to upload explicitly. Here\'s [a quick example in 
PyTorch](http://wandb.me/pytorch-colab) that does not log images. ### How do I log a PNG? 
[`wandb.Image`](../../../ref/python/data-types/image.md) converts `numpy` arrays or instances of `PILImage` to PNGs
by default. ```python wandb.log({"example": wandb.Image(...)}) # Or multiple images wandb.log({"example": 
[wandb.Image(...) for img in images]}) ``` ### How do I log a video? Videos are logged using the 
[`wandb.Video`](../../../ref/python/data-types/video.md) data type: ```python wandb.log({"example": 
wandb.Video("myvideo.mp4")}) ``` Now you can view videos in the media browser. Go to your project workspace, run 
workspace, or report and click "Add visualization" to add a rich media panel. ### How do I navigate and zoom in 
point clouds? You can hold control and use the mouse to move around inside the space. ###',
    'score': 0.23904511486347035
}

{
    'source': 'guides/integrations/huggingface.md',
    'text': "manage model lifecycle, facilitate easy tracking and auditing throughout the ML lifecyle, and 
[automate](https://docs.wandb.ai/guides/models/automation) downstream actions with webhooks or jobs. See the [Model
Registry](../model_registry) documentation for how to link a model Artifact to the Model Registry. ### 5) Visualise
evaluation outputs during training Visualing your model outputs during training or evaluation is often essential to
really understand how your model is training. By using the callbacks system in the Transformers Trainer, you can 
log additional helpful data to W&B such as your models' text generation outputs or other predictions to W&B Tables.
See the **[Custom logging section](#custom-logging-log-and-view-evaluation-samples-during-training)** below for a 
full guide on how to log evaluation outupts while training to log to a W&B Table like this: ![Shows a W&B Table 
with evaluation outputs](/images/integrations/huggingface_eval_tables.png) ### 6) Finish your W&B Run (Notebook 
only) If your training is encapsulated in a Python script, the W&B run will end when your script finishes. If you 
are using a Jupyter or Google Colab notebook, you'll need to tell us when you're done with training by calling 
`wandb.finish()`. ```python trainer.train() # start training and logging to W&B # post-training analysis, testing, 
other logged code wandb.finish() ``` ### 7) Visualize your results Once you have logged your training results you 
can explore your results dynamically in the [W&B Dashboard](../track/app.md). It's easy to compare across dozens of
runs at once, zoom in on interesting findings, and coax insights out of complex data with flexible, interactive 
visualizations. ## Advanced features and FAQs ### How do I save the best model? If `load_best_model_at_end=True` is
set in the `TrainingArguments` that are passed to the `Trainer`, then W&B will save the best performing model 
checkpoint to Artifacts. If you'd like to centralize all your best model versions across your team to organize them
by ML task, stage them for production,",
    'score': 0.2328486885623663
}

## Generating a response

There are two components of any RAG pipeline - a `Retriever` and a `ResponseGenerator`. Earlier, we designed a simple retriever. Here we are designing a simple `ResponseGenerator`.

The `generate_response` method takes the user question along with the retrieved context (chunks) as inputs and makes a LLM call using the `model` and `prompt` (system prompt). This way the generated answer is grounded on the documentation (our usecase). In this course we are using Cohere's `command-r` model.

As earlier, we have wrapped this `ResponseGenerator` class with weave for tracking the inputs and the output.

In [23]:
from scripts.response_generator import SimpleResponseGenerator

display_source(SimpleResponseGenerator)

   1 class SimpleResponseGenerator(weave.Model):                                                                   
   2     """                                                                                                       
   3     A simple response generator model using Cohere's API.                                                     
   4                                                                                                               
   5     Attributes:                                                                                               
   6         model (str): The model name to be used for generating responses.                                      
   7         prompt (str): The prompt to be used for generating responses.                                         
   8         client (cohere.ClientV2): The Cohere client for interacting with the Cohere API.                      
   9     """                                                                                                       
  10                                                                                                               
  11     model: str                                                                                                
  12     prompt: str                                                                                               
  13     client: cohere.ClientV2 = None                                                                            
  14                                                                                                               
  15     def __init__(self, **kwargs):                                                                             
  16         """                                                                                                   
  17         Initialize the SimpleResponseGenerator with the provided keyword arguments.                           
  18         Sets up the Cohere client using the API key from environment variables.                               
  19         """                                                                                                   
  20         super().__init__(**kwargs)                                                                            
  21         self.client = cohere.ClientV2(                                                                        
  22             api_key=os.environ["CO_API_KEY"],                                                                 
  23         )                                                                                                     
  24                                                                                                               
  25     @weave.op()                                                                                               
  26     def generate_context(self, context: List[Dict[str, any]]) -> List[Dict[str, any]]:                        
  27         """                                                                                                   
  28         Generate a list of contexts from the provided context list.                                           
  29                                                                                                               
  30         Args:                                                                                                 
  31             context (List[Dict[str, any]]): A list of dictionaries containing context data.                   
  32                                                                                                               
  33         Returns:                                                                                              
  34             List[Dict[str, any]]: A list of dictionaries with 'source' and 'text' keys.                       
  35         """                                        

Below is the system prompt. Consider this to be set of instructions on what to do with the user question and the retrieved contexts. In practice, the system prompt can be very detailed and involved (depending on the usecase) but we are showing a simple prompt. Later we will iterate on it and show how improving the system prompt improves the quality of the generated response.

In [24]:
INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
print(INITIAL_PROMPT)

Answer to the following question about W&B. Provide an helful and complete answer based only on the provided 
documents.

Let's generate the response for the question "How do I use Weights & Biases to log metrics in my training script?". We have already retrieved the context in the previous section and passing both the question and the context to the `generate_response` method.

In [26]:
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
answer = response_generator.generate_response(query, search_results)
print(answer)

🍩 https://wandb.ai/elshehawy/rag-course/r/call/0192cfa9-7dcd-7092-ae99-8d501122ef3c


You can use the W&B API in your training script by doing the following:
1. Import the necessary modules, including `Tabs` and `TabItem`, into your training script.
2. Call `wandb.init()` at the beginning of your script. This creates a run object on W&B servers and starts a 
separate process to stream and collect metrics.
3. Write your script as normal. When you want to log metrics, call `wandb.log()` with the metrics you want to log 
as keys and their values. For example, `wandb.log({"epoch": epoch, "val_acc": 0.94})`.
4. If you want to turn off streaming to W&B servers, run `wandb off` in your training directory or set the 
`WANDB_MODE` environment variable to `offline`.

## Simple Retrieval Augmented Generation (RAG) Pipeline

Finally, we will bring everything together. As stated earlier, a RAG pipeline primarily consists of a retriever and a response generator.

We define a class `SimpleRAGPipeline` which combines the steps of retrieval and response generation.

We'll define a `predict` method that takes the user query, retrieves relevant context using the retriever and finally synthesizes a response using the response generator.

We'll also define a few convinence methods to format the documents retrieved from the retriever and create a system prompt for the response generator.

In [27]:
from scripts.rag_pipeline import SimpleRAGPipeline

display_source(SimpleRAGPipeline)

   1 class SimpleRAGPipeline(weave.Model):                                                                         
   2     """                                                                                                       
   3     A simple RAG (Retrieval-Augmented Generation) pipeline.                                                   
   4                                                                                                               
   5     Attributes:                                                                                               
   6         retriever (weave.Model): The model used for retrieving relevant documents.                            
   7         response_generator (weave.Model): The model used for generating responses.                            
   8         top_k (int): The number of top documents to retrieve.                                                 
   9     """                                                                                                       
  10                                                                                                               
  11     retriever: weave.Model = None                                                                             
  12     response_generator: weave.Model = None                                                                    
  13     top_k: int = 5                                                                                            
  14                                                                                                               
  15     @weave.op()                                                                                               
  16     def predict(self, query: str):                                                                            
  17         """                                                                                                   
  18         Predicts a response based on the input query.                                                         
  19                                                                                                               
  20         Args:                                                                                                 
  21             query (str): The input query string.                                                              
  22                                                                                                               
  23         Returns:                                                                                              
  24             The generated response based on the retrieved context.                                            
  25         """                                                                                                   
  26         context = self.retriever.predict(query, self.top_k)                                                   
  27         return self.response_generator.predict(query, context)                                                
  28                                                                                                               

Let us initialize the `RAGPipeline`.

In [28]:
# Initialize the response generator
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)

# Bring them together as a RAG pipeline
rag_pipeline = SimpleRAGPipeline(
    retriever=retriever, response_generator=response_generator, top_k=5
)

In [29]:
response = rag_pipeline.predict("How do I get get started with wandb?")
print(response, sep="\n")

🍩 https://wandb.ai/elshehawy/rag-course/r/call/0192cfab-8064-7013-8313-db233a5528a3


To get started with W&B, you can use the following code snippet to install W&B and link your account:
```
!pip install wandb -qqq import wandb wandb.login()
```
This code can be run in a Jupyter notebook to get interactive visualizations without leaving the notebook. After 
running `wandb.init()', start a new cell with `%%wandb` to see live graphs.

You can also call `wandb.init()' from your training script to create a run object on W&B servers. A separate 
process is started to stream and collect metrics, which keeps all threads and logic out of your primary process.

Click on the link starting with a 🍩. This is the trace timeline for all the executions that happened in our simple RAG application. Go to the link and drill down to find everything that got tracked.

![weave trace timeline](../images/01_weave_trace_timeline.png)

## Key Takeaways

In this chapter, we've built a simple RAG pipeline from scratch. Here's what we've learned:

- **Data Processing**: How to ingest, chunk, and clean data using W&B Artifacts and Weave
- **Retrieval**: Implementing a basic TF-IDF based retriever
- **Response Generation**: Using Cohere's API and `command-r` model to generate responses based on retrieved context
- **RAG Pipeline**: Combining retrieval and generation into a cohesive system
- **Logging and Tracking**: Utilizing W&B Weave for efficient experiment tracking